<a href="https://colab.research.google.com/github/EmilioMobile/Machine-Learning-10x_ml_dev/blob/master/mxnet%20LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#!pip install numpy -U
#!pip install pillow -U
!pip install mxnet
!pip install gluon
!pip install gluonbook

    100% |████████████████████████████████| 337kB 13.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/3f/f3/cd/e115ddd372a84c2356d47404155a5865925013a462853a1ed7
Successfully built gluonbook
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.15
    Uninstalling prompt-toolkit-1.0.15:
      Successfully uninstalled prompt-toolkit-1.0.15


In [0]:
import mxnet as mx
#import gluon as gb
import gluonbook as gb
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time

#### Model, LeNet

In [0]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, padding=2, activation='sigmoid'),
       nn.AvgPool2D(pool_size=2, strides=2), 
       nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
       nn.AvgPool2D(pool_size=2, strides=2),
       nn.Dense(120, activation='sigmoid'),
       nn.Dense(84, activation='sigmoid'),
       nn.Dense(10))

#### Initialize

In [49]:
X = nd.random.uniform(1,1, shape=(1,1,28, 28))
net.initialize(force_reinit=True)
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t', X.shape)

conv5 output shape:	 (1, 6, 28, 28)
pool3 output shape:	 (1, 6, 14, 14)
conv6 output shape:	 (1, 16, 10, 10)
pool4 output shape:	 (1, 16, 5, 5)
dense3 output shape:	 (1, 120)
dense4 output shape:	 (1, 84)
dense5 output shape:	 (1, 10)


#### Data Acquisition and Training


In [50]:
batch_size = 256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size=batch_size)

In [37]:
def try_gpu4():
  try:
    ctx = mx.gpu()
    _ = nd.zeros((1,), ctx=ctx)
  except mx.base.MXNetError:
    ctx = mx.cpu()
  return ctx

ctx = try_gpu4()
ctx

cpu(0)

In [0]:
def evaluate_accuracy(data_iter, net, ctx):
  acc = nd.array([0], ctx=ctx)
  for X, y in data_iter:
    # if ctx is the GPU, copy the data to the GPU
    X, y = X.as_in_context(ctx), y.as_in_context(ctx)
    acc += gb.accuracy(net(X), y)
  return acc.asscalar() / len(data_iter)

In [0]:
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
  print('training on', ctx)
  loss = gloss.SoftmaxCrossEntropyLoss()
  for epoch in range(num_epochs):
    train_l_sum, train_acc_sum, start = 0, 0, time.time()
    for X, y in train_iter:
      X, y = X.as_in_context(ctx), y.as_in_context(ctx)
      with autograd.record():
        y_hat = net(X)
        l = loss(y_hat, y)
      l.backward()
      trainer.step(batch_size)
      train_l_sum += l.mean().asscalar()
      train_acc_sum += gb.accuracy(y_hat, y)
      test_acc = evaluate_accuracy(test_iter, net, ctx)
      print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
            'time %.1f sec' % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / len(train_iter), test_acc, time.time() - start))

In [41]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5 =(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

NameError: ignored